In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Fix me: Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelterOperations import AnimalShelter


# Connect to database via CRUD Module
db = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Fix me: Add in Grazioso Salvare’s logo
image_filename = 'GraziosoLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Hr(),
    html.Div(
        children=[
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water Rescue'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                    {'label': 'Reset', 'value': 'Reset'}
                ],
                value='Reset'
            )
        ]
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_size=10,  # Set page size for better usability
                         style_table={'overflowX': 'auto'},
                         style_data={'whiteSpace': 'normal', 'height': 'auto'},
                         sort_action='native'
                        ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ]),
    html.Footer(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '75px', 'height': '75px'}), href='https://www.snhu.edu'), style={'textAlign': 'center'}),
    html.Footer('Created by Brayden Massa', style={'textAlign': 'center'})
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        data = df.to_dict('records')
    else:
        query = {}
        if filter_type == 'Water Rescue':
            query['breed'] = {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}
        elif filter_type == 'Mountain or Wilderness Rescue':
            query['breed'] = {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}
        elif filter_type == 'Disaster or Individual Tracking':
            query['breed'] = {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}
        filtered_df = pd.DataFrame.from_records(db.read(query))
        filtered_df.drop(columns=['_id'], inplace=True)
        data = filtered_df.to_dict('records')
    return data

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return
    df_view = pd.DataFrame.from_dict(viewData)
    if not df_view.empty:
        fig = px.pie(df_view, names='breed', title='Preferred Breeds')
        return [dcc.Graph(figure=fig)]
    return []

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return
    dff = pd.DataFrame.from_dict(viewData)
    if index:
        row = index[0]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                    dl.Tooltip(dff.iloc[row, 4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row, 9])
                    ])
                ])
            ])
        ]
    return []

app.run_server(debug=True)


Dash app running on http://127.0.0.1:19592/
